In [1]:
import tensorflow as tf
from tensorflow import keras
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [21]:
import pandas as pd
import numpy as np
from numpy import mean
import matplotlib.pyplot as plt
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose ,
                                     BatchNormalization, LeakyReLU, Activation, Dropout, 
                                     Embedding, Concatenate) 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import backend

from tensorflow.keras.utils import plot_model
from numpy.random import random
from numpy.random import choice
 
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.models import load_model
from matplotlib import pyplot

In [3]:
def load_real_samples():
    train1 = pd.read_csv("/home/eh2/Downloads/Thesis/Dataset/Character & DIgit/malechar1.csv")
    train2 = pd.read_csv("/home/eh2/Downloads/Thesis/Dataset/Character & DIgit/malechar2.csv")

    frames = [train1, train2]

    train = pd.concat(frames, ignore_index= True)

    X_train = train["label"] - 10
    Y_train = train.drop(labels = ["label"],axis = 1)
    Y_train = (Y_train - 127.5) / 127.5

    Y_train = Y_train.values.reshape(-1,28,28,1)

    return [Y_train, X_train]

In [4]:
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)

In [5]:
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [6]:
def define_discriminator(in_shape=(28,28,1), n_classes=50):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	const = ClipConstraint(0.01)
	# image input
	in_image = Input(shape=in_shape)
	
	# downsample to 14x14
	fe = Conv2D(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const)(in_image)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.5)(fe)
	# normal
	fe = Conv2D(64, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(fe)
	fe = BatchNormalization()(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.5)(fe)
	# downsample to 7x7
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const)(fe)
	fe = BatchNormalization()(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.5)(fe)
	# normal
	fe = Conv2D(256, (3,3), padding='same', kernel_initializer=init, kernel_constraint=const)(fe)
	fe = BatchNormalization()(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.5)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# real/fake output
	out1 = Dense(1, activation='linear')(fe)
	# class label output
	out2 = Dense(n_classes, activation='softmax')(fe)
	# define model
	model = Model(in_image, [out1, out2])
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=[wasserstein_loss, 'sparse_categorical_crossentropy'], optimizer=opt)
	return model

In [7]:
def define_generator(latent_dim, n_classes=50):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 7 * 7
	li = Dense(n_nodes, kernel_initializer=init)(li)
	# reshape to additional channel
	li = Reshape((7, 7, 1))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	n_nodes = 384 * 7 * 7
	gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
	gen = Activation('relu')(gen)
	gen = Reshape((7, 7, 384))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 14x14
	gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
	gen = BatchNormalization()(gen)
	gen = Activation('relu')(gen)
	# upsample to 28x28
	gen = Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
	out_layer = Activation('tanh')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model

In [8]:
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect the outputs of the generator to the inputs of the discriminator
	gan_output = d_model(g_model.output)
	# define gan model as taking noise and label and outputting real/fake and label outputs
	model = Model(g_model.input, gan_output)
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=[wasserstein_loss, 'sparse_categorical_crossentropy'], optimizer=opt)
	return model

In [9]:
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = -ones((n_samples, 1))
	return [X, labels], y

In [10]:
def generate_latent_points(latent_dim, n_samples, n_classes=50):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

In [11]:
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = ones((n_samples, 1))
	return [images, labels_input], y

In [12]:
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	[X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(100):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = 'generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

In [25]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=1000, n_batch=64, n_critic=5):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	e=0
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	d_r1_hist, d_r2_hist, g_1_hist,g_2_hist, d_f1_hist, d_f2_hist = list(), list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		d_r1_tmp, d_r2_tmp, d_f1_tmp, d_f2_tmp = list(), list(), list(), list()
		for _ in range(n_critic):	
			# get randomly selected 'real' samples
			[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			_,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
			d_r1_tmp.append(d_r1)
			d_r2_tmp.append(d_r2)
			# generate 'fake' examples
			[X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			_,d_f1,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
		d_r1_hist.append(mean(d_r1_tmp))
		d_r2_hist.append(mean(d_r2_tmp))
		d_f1_hist.append(d_f1_tmp)
		d_f1_hist.append(d_f2_tmp)
		# prepare points in latent space as input for the generator
		[z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = -ones((n_batch, 1))
		# update the generator via the discriminator's error
		_,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
		# summarize loss on this batch
		g_1_hist.append(g_1)
		g_2_hist.append(g_2)
		# evaluate the model performance every 'epoch'
		if (i+1) % (bat_per_epo * 10) == 0:
			e+=1
			print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (e, d_r1,d_r2, d_f1,d_f2, g_1,g_2))
			summarize_performance(e, g_model, latent_dim)

In [19]:
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()

In [26]:
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, dr[-15582.478,4.530], df[-9622.639,8.092], g[14605.562,8.226]
>Saved: generated_plot_0002.png and model_0002.h5
>2, dr[-24416.645,5.635], df[-13762.965,10.219], g[47824.449,10.724]
>Saved: generated_plot_0003.png and model_0003.h5
>3, dr[-28869.902,7.949], df[-55819.613,14.877], g[51810.574,12.034]
>Saved: generated_plot_0004.png and model_0004.h5
>4, dr[-39775.797,11.223], df[-24542.949,15.839], g[49259.445,17.929]
>Saved: generated_plot_0005.png and model_0005.h5
>5, dr[-89474.688,8.975], df[-17904.133,16.068], g[66437.969,17.764]
>Saved: generated_plot_0006.png and model_0006.h5
>6, dr[-34245.695,11.057], df[-31491.139,22.101], g[52379.391,19.977]
>Saved: generated_plot_0007.png and model_0007.h5
>7, dr[-90637.867,12.180], df[-121120.781,20.488], g[74864.203,22.932]
>Saved: generated_plot_0008.png and model_0008.h5
>8, dr[-123565.000,12.799], df[-45980.699,25.972], g[50187.289,26.635]
>Saved: generated_plot_0009.png and model_0009.h5
>9, dr[-150460.688,12.513], df[110720.211,25.

>Saved: generated_plot_0068.png and model_0068.h5
>68, dr[-6980038.000,95.183], df[7015597.000,92.646], g[-6941189.000,91.220]
>Saved: generated_plot_0069.png and model_0069.h5
>69, dr[-7006501.000,85.453], df[7024484.000,87.146], g[-7022377.500,92.637]
>Saved: generated_plot_0070.png and model_0070.h5
>70, dr[-7110214.000,74.857], df[7075833.000,73.856], g[-7072722.500,82.345]
>Saved: generated_plot_0071.png and model_0071.h5
>71, dr[-7180133.000,74.011], df[7175164.000,93.434], g[-7155786.000,89.946]
>Saved: generated_plot_0072.png and model_0072.h5
>72, dr[-7181158.000,93.431], df[7185671.500,81.004], g[-7170100.000,92.749]
>Saved: generated_plot_0073.png and model_0073.h5
>73, dr[-7222666.000,76.343], df[7252768.000,96.372], g[-7258739.000,86.104]
>Saved: generated_plot_0074.png and model_0074.h5
>74, dr[-7291943.000,117.200], df[7332863.000,96.317], g[-7279911.000,94.905]
>Saved: generated_plot_0075.png and model_0075.h5
>75, dr[-7350365.000,99.452], df[7326096.000,116.232], g[-73

In [3]:
print(d_r1_hist)

NameError: name 'd_r1_hist' is not defined